# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../WeatherPY/weather_data.csv')
weather_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Roseburg,43.22,-123.34,89.60,24,1,10.29,US,1592618965
1,1,Souillac,-20.52,57.52,71.60,68,40,14.99,MU,1592618967
2,2,Peña,12.09,123.94,83.86,72,0,8.63,PH,1592618968
3,3,Havre-St-Pierre,50.23,-63.60,51.80,93,90,14.99,CA,1592618969
4,4,Bengkulu,-3.80,102.27,80.80,71,44,1.01,ID,1592618804


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#drop na values
weather_df = weather_df.dropna()
#define location and weight
weight = weather_df["Humidity"].astype(float)
locations = weather_df[['Lat', 'Lng']].astype(float)
#configure maps
gmaps.configure(api_key=g_key)
fig_layout={'width': '1000px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=fig_layout,zoom_level=1,center=(0,0))


heat_layer = gmaps.heatmap_layer(locations, weights=weight,max_intensity=max(weather_df["Humidity"]),point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
new_weather_df = weather_df
#A max temperature lower than 80 degrees but higher than 70.
new_weather_df= new_weather_df.loc[(new_weather_df["Max Temp"] < 80) & (new_weather_df["Max Temp"] > 70)]
#Wind speed less than 10 mph.
new_weather_df=new_weather_df.loc[(new_weather_df["Wind Speed"] < 10)]
#Zero cloudiness.
new_weather_df=new_weather_df.loc[(new_weather_df["Cloudiness"] ==0)]

#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

#making sure there are less than 10 locations-- found 9 locations
#locations_count=new_weather_df["Lat"].nunique()

new_weather_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,37,Chardara,41.25,67.97,76.57,28,0,2.62,KZ,1592619006
49,49,Mount Isa,-20.73,139.50,78.80,34,0,8.05,AU,1592619019
65,65,Hohhot,40.81,111.65,78.80,41,0,6.71,CN,1592619036
69,69,Pimenteiras,-6.25,-41.42,70.16,88,0,4.21,BR,1592619041
155,155,São Filipe,14.90,-24.50,76.62,79,0,7.83,CV,1592619143
236,236,Verkhnevilyuysk,63.45,120.32,78.89,38,0,3.89,RU,1592619239
254,254,Jalu,29.03,21.55,78.66,25,0,8.08,LY,1592619263
299,299,Mossamedes,-15.20,12.15,70.57,56,0,3.15,AO,1592619320
410,410,Yangi Marg`ilon,40.43,71.72,71.60,33,0,9.17,UZ,1592619457
444,444,Marfino,55.70,37.38,72.00,82,0,4.47,RU,1592619496


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df=new_weather_df
#set up parameters
params = {
    "radius": 5000,
    "ke word":"hotel",
    "types": "lodging",
    "key": g_key
}

# find lat lng in hotel df to get location in parameters
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each time but leave the other paramets the same
    params["location"] = f"{lat},{lng}"

    # make API request and get results in json format
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel = requests.get(base_url, params=params)
    hotel=hotel.json()
    #pprint(hotel)

   
    # add hotel name to hotel dataframe.there may not be hotels nearby so need skip when missing value occurs.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    
    except (KeyError, IndexError):
        #print("No nearby hotel... skipping.")
        print("")
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
37,37,Chardara,41.25,67.97,76.57,28,0,2.62,KZ,1592619006,Жанаргул
49,49,Mount Isa,-20.73,139.50,78.80,34,0,8.05,AU,1592619019,ibis Styles Mt Isa Verona
65,65,Hohhot,40.81,111.65,78.80,41,0,6.71,CN,1592619036,Sheraton Hohhot Hotel
69,69,Pimenteiras,-6.25,-41.42,70.16,88,0,4.21,BR,1592619041,Hotel Central
155,155,São Filipe,14.90,-24.50,76.62,79,0,7.83,CV,1592619143,Tortuga B&B
236,236,Verkhnevilyuysk,63.45,120.32,78.89,38,0,3.89,RU,1592619239,Gostinitsa Mila
254,254,Jalu,29.03,21.55,78.66,25,0,8.08,LY,1592619263,Jalu Hotel
299,299,Mossamedes,-15.20,12.15,70.57,56,0,3.15,AO,1592619320,Hotel Chik Chik Namibe
410,410,Yangi Marg`ilon,40.43,71.72,71.60,33,0,9.17,UZ,1592619457,Ikathouse
444,444,Marfino,55.70,37.38,72.00,82,0,4.47,RU,1592619496,Kardiologicheskiy Sanatornyy Tsentr Peredelkino


In [12]:

# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
fig_layout={'width': '1000px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=fig_layout,zoom_level=2,center=(0,0))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig



Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…